In [1]:
from pymongo import MongoClient
import pandas as pd
import numpy as np


In [2]:
client = MongoClient('mongodb://127.0.0.1:27017/')
db = client.reddit

In [3]:
collection = db.data

In [4]:
df = pd.DataFrame(list(collection.find()))

In [5]:
del df['_id']

In [6]:
df.head()

,author,body,comments,flair,numComs,score,time,title,url
0,parvdave,I don't understand what difference it will mak...,This post has been marked [R]. Please help ke...,AskIndia,1243,505,2016-11-09 05:03:02,Eli5: What the abolition of 500 and 1000 rupee...,https://www.reddit.com/r/india/comments/5btv1v...
1,alphasayen,What are the best alternatives to condoms that...,Abstinence.\n\n(Cries virgin tears) Pull out ...,AskIndia,30,17,2019-07-20 21:36:47,What is the best alternative to condoms that y...,https://www.reddit.com/r/india/comments/cfjf1z...
2,Sudroy,Anyone knows where to watch eagoler chokh and ...,"If it's aschhe abar shobor, then that's avail...",AskIndia,7,7,2019-07-21 00:30:47,Bengali movies,https://www.reddit.com/r/india/comments/cfkmjb...
3,bhayanak-aatma,This question is for all but more specifically...,Takeshi's castle in Javed Jaffrey voice. Mad...,AskIndia,70,21,2019-07-08 02:39:43,What was your favourite TV serials growing up?,https://www.reddit.com/r/india/comments/ca6nqa...
4,LifeWastedOnLiving,Given how there are different Career expectati...,[deleted] Have done that.\nRelocated to anoth...,AskIndia,49,59,2019-07-16 01:30:24,Survey : How many of you guys(men) would fully...,https://www.reddit.com/r/india/comments/cdgkis...


In [7]:
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from sklearn.pipeline import Pipeline

In [8]:
df =df.replace(np.nan, '', regex=True)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2649 entries, 0 to 2648
Data columns (total 9 columns):
author      2649 non-null object
body        2649 non-null object
comments    2649 non-null object
flair       2649 non-null object
numComs     2649 non-null object
score       2649 non-null object
time        2649 non-null object
title       2649 non-null object
url         2649 non-null object
dtypes: object(9)
memory usage: 186.3+ KB


In [10]:
df.keys()

Index(['author', 'body', 'comments', 'flair', 'numComs', 'score', 'time',
       'title', 'url'],
      dtype='object')

In [11]:
label = df.flair
fTitle = df.title
fBody = df.body
fAuthor = df.author
fNumComs = df.numComs
fscore = df.score
fComments = df.comments

In [12]:
xTrainTitle,xTestTitle,yTrainTitle,yTestTitle=train_test_split(fTitle,label,test_size=0.33)

xTrainBody,xTestBody,yTrainBody,yTestBody=train_test_split(fBody,label,test_size=0.33)

xTrainAuthor,xTestAuthor,yTrainAuthor,yTestAuthor=train_test_split(fAuthor,label,test_size=0.33)

xTrainNum,xTestNum,yTrainNum,yTestNum=train_test_split(fNumComs,label,test_size=0.33)

xTrainComments,xTestComments,yTrainComments,yTestComments=train_test_split(fComments,label,test_size=0.33)

In [13]:
encoder = preprocessing.LabelEncoder()
yTrainTitle=encoder.fit_transform(yTrainTitle)
yTestTitle = encoder.fit_transform(yTestTitle)

yTrainBody=encoder.fit_transform(yTrainBody)
yTestBody = encoder.fit_transform(yTestBody)

yTrainAuthor=encoder.fit_transform(yTrainAuthor)
yTestAuthor = encoder.fit_transform(yTestAuthor)

yTrainComments=encoder.fit_transform(yTrainComments)
yTestComments = encoder.fit_transform(yTestComments)

In [14]:
count_vect = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',stop_words='english')
count_vect.fit(fTitle)
xTrct = count_vect.transform(xTrainTitle)
xTect = count_vect.transform(xTestTitle)

count_vect = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',stop_words='english')
count_vect.fit(fBody)
xTrcb = count_vect.transform(xTrainBody)
xTecb = count_vect.transform(xTestBody)

count_vect = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}')
count_vect.fit(fAuthor)
xTrca = count_vect.transform(xTrainAuthor)
xTeca = count_vect.transform(xTestAuthor)

count_vect = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',stop_words='english')
count_vect.fit(fComments)
xTrcc = count_vect.transform(xTrainComments)
xTecc = count_vect.transform(xTestComments)


In [15]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid,yTest, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    global Y_test
    return metrics.accuracy_score(predictions,yTest)

In [16]:
accuracy = train_model(naive_bayes.MultinomialNB(), xTrct, yTrainTitle, xTect,yTestTitle)
print("Title NB, Count Vectors: ", accuracy)

accuracy = train_model(naive_bayes.MultinomialNB(), xTrcb, yTrainBody, xTecb,yTestBody)
print("Body NB, Count Vectors: ", accuracy)

accuracy = train_model(naive_bayes.MultinomialNB(), xTrcc, yTrainComments, xTecc,yTestComments)
print("Comments, Count Vectors: ", accuracy)

accuracy = train_model(naive_bayes.MultinomialNB(), xTrca, yTrainAuthor, xTeca,yTestAuthor)
print("Author, Count Vectors: ", accuracy)


Title NB, Count Vectors:  0.4937142857142857
Body NB, Count Vectors:  0.16228571428571428
Comments, Count Vectors:  0.376
Author, Count Vectors:  0.288


In [17]:
# linear_model.LogisticRegression()
accuracy = train_model(linear_model.LogisticRegression(), xTrct, yTrainTitle, xTect,yTestTitle)
print("Title NB, Count Vectors: ", accuracy)

accuracy = train_model(linear_model.LogisticRegression(), xTrcb, yTrainBody, xTecb,yTestBody)
print("Body NB, Count Vectors: ", accuracy)

accuracy = train_model(linear_model.LogisticRegression(), xTrcc, yTrainComments, xTecc,yTestComments)
print("Comments, Count Vectors: ", accuracy)

accuracy = train_model(linear_model.LogisticRegression(), xTrca, yTrainAuthor, xTeca,yTestAuthor)
print("Author, Count Vectors: ", accuracy)

Title NB, Count Vectors:  0.47085714285714286


c:\users\utsavs~1\desktop\reddit~1\myenv\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\utsavs~1\desktop\reddit~1\myenv\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Body NB, Count Vectors:  0.2057142857142857
Comments, Count Vectors:  0.448
Author, Count Vectors:  0.30857142857142855


In [18]:
accuracy = train_model(ensemble.RandomForestClassifier(), xTrct, yTrainTitle, xTect,yTestTitle)
print("Title NB, Count Vectors: ", accuracy)

accuracy = train_model(ensemble.RandomForestClassifier(), xTrcb, yTrainBody, xTecb,yTestBody)
print("Body NB, Count Vectors: ", accuracy)

accuracy = train_model(ensemble.RandomForestClassifier(), xTrcc, yTrainComments, xTecc,yTestComments)
print("Comments, Count Vectors: ", accuracy)

accuracy = train_model(ensemble.RandomForestClassifier(), xTrca, yTrainAuthor, xTeca,yTestAuthor)
print("Author, Count Vectors: ", accuracy)

c:\users\utsavs~1\desktop\reddit~1\myenv\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Title NB, Count Vectors:  0.38171428571428573
Body NB, Count Vectors:  0.22057142857142858


c:\users\utsavs~1\desktop\reddit~1\myenv\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\utsavs~1\desktop\reddit~1\myenv\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Comments, Count Vectors:  0.29942857142857143
Author, Count Vectors:  0.3017142857142857


c:\users\utsavs~1\desktop\reddit~1\myenv\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [19]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, confusion_matrix

In [20]:
df['TC']= df.title.str.cat(df['comments'].astype('str'),sep = " ")
df['TCB'] = df.TC.str.cat(df['body'].astype('str'),sep = " ")
df['TCBC'] = df.TCB.str.cat(df['time'].astype('str'),sep=" ")
df['TCBU']=df.TCB.str.cat(df.url.astype('str'),sep = " ")
df['TCBUC']=df.TCBU.str.cat(df.time.astype('str'),sep=" ")
df['TCBUA']=df.TCBU.str.cat(df.author.astype('str'),sep = " ")


In [21]:
fTCB = df.TCB
fTC = df.TC
fTCBU = df.TCBU
fTCBUA = df.TCBUA
fTCBC = df.TCBC

In [22]:
flairs = ["AskIndia", "Non-Political", "[R]eddiquette", "Scheduled", "Photography", "Science/Technology", "Politics", "Business/Finance", "Policy/Economy", "Sports", "Food"]

# Title Comment Body Created

In [ ]:
lr = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', LogisticRegression()),
              ])

lr2 = Pipeline([('vect', TfidfVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', LogisticRegression()),
              ])



nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])

nb2 = Pipeline([('vect', TfidfVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])

rf =  Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', RandomForestClassifier()),
              ])

rf2 = Pipeline([('vect', TfidfVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', RandomForestClassifier()),
              ])


In [23]:
xTrtcbu,xTetcbu,yTrtcbu,yTetcbu=train_test_split(fTCBC,label,test_size=0.25)

LR = lr.fit(xTrtcbu,y=yTrtcbu)
LR2 = lr2.fit(xTrtcbu,y=yTrtcbu)


NB = nb.fit(xTrtcbu,y=yTrtcbu)
NB2 = nb2.fit(xTrtcbu,y=yTrtcbu)

RF = rf.fit(xTrtcbu,y=yTrtcbu)
RF2 = rf2.fit(xTrtcbu,y=yTrtcbu)


flairs = ["AskIndia", "Non-Political", "[R]eddiquette", "Scheduled", "Photography", "Science/Technology", "Politics", "Business/Finance", "Policy/Economy", "Sports", "Food"]

y_pred = lr.predict(xTetcbu)
y_pred2 = lr2.predict(xTetcbu)

print('LOG accuracy %s' % accuracy_score(y_pred, yTetcbu))
print(classification_report(yTetcbu, y_pred,target_names=flairs))

print('LOG accuracy LR2 %s' % accuracy_score(y_pred2, yTetcbu))
print(classification_report(yTetcbu, y_pred2,target_names=flairs))

y_pred = nb.predict(xTetcbu)
y_pred2 = nb2.predict(xTetcbu)

print('LOG accuracy %s' % accuracy_score(y_pred, yTetcbu))
print(classification_report(yTetcbu, y_pred,target_names=flairs))

print('LOG accuracy nb22 %s' % accuracy_score(y_pred2, yTetcbu))
print(classification_report(yTetcbu, y_pred2,target_names=flairs))

y_pred = rf.predict(xTetcbu)
y_pred2 = rf2.predict(xTetcbu)

print('LOG accuracy %s' % accuracy_score(y_pred, yTetcbu))
print(classification_report(yTetcbu, y_pred,target_names=flairs))

print('LOG accuracy RF2 %s' % accuracy_score(y_pred2, yTetcbu))
print(classification_report(yTetcbu, y_pred2,target_names=flairs))

c:\users\utsavs~1\desktop\reddit~1\myenv\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\utsavs~1\desktop\reddit~1\myenv\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
c:\users\utsavs~1\desktop\reddit~1\myenv\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\utsavs~1\desktop\reddit~1\myenv\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
c:\users\utsavs~1\desktop\reddit~1\myenv\l

LOG accuracy 0.5867269984917044
                    precision    recall  f1-score   support

          AskIndia       0.53      0.59      0.56        69
     Non-Political       0.61      0.36      0.45        70
     [R]eddiquette       0.82      0.78      0.80        46
         Scheduled       0.33      0.41      0.36        64
       Photography       0.70      0.89      0.78        45
Science/Technology       0.49      0.64      0.56        61
          Politics       0.50      0.75      0.60        65
  Business/Finance       0.80      0.85      0.83        48
    Policy/Economy       0.64      0.47      0.54        60
            Sports       0.88      0.89      0.89        57
              Food       0.39      0.17      0.23        78

          accuracy                           0.59       663
         macro avg       0.61      0.62      0.60       663
      weighted avg       0.59      0.59      0.57       663

LOG accuracy LR2 0.5882352941176471
                    precision

c:\users\utsavs~1\desktop\reddit~1\myenv\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


LOG accuracy 0.3438914027149321
                    precision    recall  f1-score   support

          AskIndia       0.22      0.41      0.29        69
     Non-Political       0.20      0.23      0.21        70
     [R]eddiquette       0.51      0.50      0.51        46
         Scheduled       0.18      0.20      0.19        64
       Photography       0.44      0.71      0.55        45
Science/Technology       0.25      0.25      0.25        61
          Politics       0.24      0.25      0.24        65
  Business/Finance       0.73      0.67      0.70        48
    Policy/Economy       0.32      0.18      0.23        60
            Sports       0.74      0.61      0.67        57
              Food       0.39      0.09      0.15        78

          accuracy                           0.34       663
         macro avg       0.39      0.37      0.36       663
      weighted avg       0.37      0.34      0.34       663

LOG accuracy RF2 0.3212669683257919
                    precision

# Title Comments Body

In [24]:
xTrtcbu,xTetcbu,yTrtcbu,yTetcbu=train_test_split(fTCB,label,test_size=0.25)
LR = lr.fit(xTrtcbu,y=yTrtcbu)
LR2 = lr2.fit(xTrtcbu,y=yTrtcbu)


NB = nb.fit(xTrtcbu,y=yTrtcbu)
NB2 = nb2.fit(xTrtcbu,y=yTrtcbu)

RF = rf.fit(xTrtcbu,y=yTrtcbu)
RF2 = rf2.fit(xTrtcbu,y=yTrtcbu)


flairs = ["AskIndia", "Non-Political", "[R]eddiquette", "Scheduled", "Photography", "Science/Technology", "Politics", "Business/Finance", "Policy/Economy", "Sports", "Food"]

y_pred = lr.predict(xTetcbu)
y_pred2 = lr2.predict(xTetcbu)

print('LOG accuracy %s' % accuracy_score(y_pred, yTetcbu))
print(classification_report(yTetcbu, y_pred,target_names=flairs))

print('LOG accuracy LR2 %s' % accuracy_score(y_pred2, yTetcbu))
print(classification_report(yTetcbu, y_pred2,target_names=flairs))

y_pred = nb.predict(xTetcbu)
y_pred2 = nb2.predict(xTetcbu)

print('LOG accuracy %s' % accuracy_score(y_pred, yTetcbu))
print(classification_report(yTetcbu, y_pred,target_names=flairs))

print('LOG accuracy nb2 %s' % accuracy_score(y_pred2, yTetcbu))
print(classification_report(yTetcbu, y_pred2,target_names=flairs))

y_pred = rf.predict(xTetcbu)
y_pred2 = rf2.predict(xTetcbu)

print('LOG accuracy %s' % accuracy_score(y_pred, yTetcbu))
print(classification_report(yTetcbu, y_pred,target_names=flairs))

print('LOG accuracy RF2 %s' % accuracy_score(y_pred2, yTetcbu))
print(classification_report(yTetcbu, y_pred2,target_names=flairs))

c:\users\utsavs~1\desktop\reddit~1\myenv\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\utsavs~1\desktop\reddit~1\myenv\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
c:\users\utsavs~1\desktop\reddit~1\myenv\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\utsavs~1\desktop\reddit~1\myenv\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
c:\users\utsavs~1\desktop\reddit~1\myenv\l

LOG accuracy 0.6093514328808446
                    precision    recall  f1-score   support

          AskIndia       0.48      0.56      0.52        77
     Non-Political       0.54      0.40      0.46        65
     [R]eddiquette       0.90      0.77      0.83        56
         Scheduled       0.46      0.32      0.38        75
       Photography       0.84      0.88      0.86        58
Science/Technology       0.40      0.68      0.50        50
          Politics       0.59      0.61      0.60        67
  Business/Finance       0.81      0.92      0.86        48
    Policy/Economy       0.61      0.57      0.59        54
            Sports       0.86      0.86      0.86        57
              Food       0.38      0.32      0.35        56

          accuracy                           0.61       663
         macro avg       0.62      0.63      0.62       663
      weighted avg       0.61      0.61      0.61       663

LOG accuracy LR2 0.6003016591251885
                    precision

# Title Comments Body Url

In [25]:
xTrtcbu,xTetcbu,yTrtcbu,yTetcbu=train_test_split(fTCBU,label,test_size=0.25)

LR = lr.fit(xTrtcbu,y=yTrtcbu)
LR2 = lr2.fit(xTrtcbu,y=yTrtcbu)


NB = nb.fit(xTrtcbu,y=yTrtcbu)
NB2 = nb2.fit(xTrtcbu,y=yTrtcbu)

RF = rf.fit(xTrtcbu,y=yTrtcbu)
RF2 = rf2.fit(xTrtcbu,y=yTrtcbu)


flairs = ["AskIndia", "Non-Political", "[R]eddiquette", "Scheduled", "Photography", "Science/Technology", "Politics", "Business/Finance", "Policy/Economy", "Sports", "Food"]

y_pred = lr.predict(xTetcbu)
y_pred2 = lr2.predict(xTetcbu)

print('LOG accuracy %s' % accuracy_score(y_pred, yTetcbu))
print(classification_report(yTetcbu, y_pred,target_names=flairs))

print('LOG accuracy LR2 %s' % accuracy_score(y_pred2, yTetcbu))
print(classification_report(yTetcbu, y_pred2,target_names=flairs))

y_pred = nb.predict(xTetcbu)
y_pred2 = nb2.predict(xTetcbu)

print('LOG accuracy %s' % accuracy_score(y_pred, yTetcbu))
print(classification_report(yTetcbu, y_pred,target_names=flairs))

print('LOG accuracy nb22 %s' % accuracy_score(y_pred2, yTetcbu))
print(classification_report(yTetcbu, y_pred2,target_names=flairs))

y_pred = rf.predict(xTetcbu)
y_pred2 = rf2.predict(xTetcbu)

print('LOG accuracy %s' % accuracy_score(y_pred, yTetcbu))
print(classification_report(yTetcbu, y_pred,target_names=flairs))

print('LOG accuracy RF2 %s' % accuracy_score(y_pred2, yTetcbu))
print(classification_report(yTetcbu, y_pred2,target_names=flairs))

c:\users\utsavs~1\desktop\reddit~1\myenv\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\utsavs~1\desktop\reddit~1\myenv\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
c:\users\utsavs~1\desktop\reddit~1\myenv\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\utsavs~1\desktop\reddit~1\myenv\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
c:\users\utsavs~1\desktop\reddit~1\myenv\l

LOG accuracy 0.6033182503770739
                    precision    recall  f1-score   support

          AskIndia       0.43      0.75      0.55        67
     Non-Political       0.63      0.35      0.45        74
     [R]eddiquette       0.81      0.71      0.76        55
         Scheduled       0.54      0.36      0.43        72
       Photography       0.76      0.92      0.83        37
Science/Technology       0.53      0.66      0.59        65
          Politics       0.50      0.58      0.54        65
  Business/Finance       0.89      0.81      0.85        58
    Policy/Economy       0.49      0.55      0.52        60
            Sports       0.78      0.92      0.84        49
              Food       0.61      0.31      0.41        61

          accuracy                           0.60       663
         macro avg       0.63      0.63      0.62       663
      weighted avg       0.62      0.60      0.59       663

LOG accuracy LR2 0.6018099547511312
                    precision